**Importing all the necessary libraries**

In [ ]:
from sqlalchemy import create_engine, MetaData, update, select
from sqlalchemy.orm import Session
import pandas as pd

**Connecting notebook to the postgres database via postgres connection string**

In [ ]:
def read_database_connection_string():
    text_file = open(r"D:/Project Hamburg/13-production/db-connection-string.txt", "r")
    CONNECTION_STRING = text_file.read()
    text_file.close()
    return CONNECTION_STRING

engine = create_engine(read_database_connection_string())
session = Session(engine)
conn = engine.connect()

**List of Tables within Database.**

In [ ]:
meta = MetaData(bind=engine)
MetaData.reflect(meta)
print("List of Tables in Postgres database.")
print(list(meta.tables.keys()))
ExchangeTable = meta.tables['Exchange']
TickerTable = meta.tables['Ticker']
TDAmeritradeDailyTable = meta.tables['TDAmeritradeDailyPrice']
TDAmeritradeMinuteTable = meta.tables['TDAmeritradeMinutePrice']
TickerFundamentalTable = meta.tables['TickerFundamental']

<ipython-input-3-51895af38c55>:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  meta = MetaData(bind=engine)


List of Tables in Postgres database.
['Exchange', 'Ticker', 'TDAmeritradeDailyPrice', 'TDAmeritradeMinutePrice', 'TickerFundamental']


**Querying database using Sqlalchemy ORM.**

In [ ]:
statement = select(ExchangeTable)

for row in conn.execute(statement):
    print(row)

(1, 'S&P500', "Standard and Poor's 500", '', 'United States of America')


**Querying database using pandas.**

**1. Exchage Table**

In [ ]:
statement = select(ExchangeTable)
df_exchange = pd.read_sql(statement, session.bind)
df_exchange.head()

,Id,Abbreviation,Name,City,Country
0,1,S&P500,Standard and Poor's 500,,United States of America


**2. Ticker Table**

In [ ]:
statement = select(TickerTable)
df_ticker = pd.read_sql(statement, session.bind)
print("Total Tickers: ", len(df_ticker))
df_ticker.head()

Total Tickers:  791


,Id,ExchangeId,Symbol,Name
0,2,1,ABT,
1,3,1,ABBV,
2,4,1,ABMD,
3,5,1,ACN,
4,6,1,ATVI,


**3. Ticker Fundamental Table**

In [ ]:
statement = select(TickerFundamentalTable)
df_ticker_fundamental = pd.read_sql(statement, session.bind)
df_ticker_fundamental.head()

,Id,TickerId,Date,MarketCapital,PERatio,EPS


**4. TDAmeritrade Daily Table**

In [ ]:
#Parameters
ticker_name = "$SPX.X"
start_date = "2022-05-10"
end_date = "2022-07-10"
order_by_date_desc = False

# Query
statement = select(TickerTable.c.Id).where(TickerTable.c.Symbol == ticker_name)
for row in conn.execute(statement):
    ticker_id = row[0]
statement = select(TDAmeritradeDailyTable)
if(ticker_name != ""):
    statement = statement.where(TDAmeritradeDailyTable.c.TickerId == ticker_id)
if(start_date != ""):
    statement = statement.where(TDAmeritradeDailyTable.c.Date >= start_date)
if(end_date != ""):
    statement = statement.where(TDAmeritradeDailyTable.c.Date <= end_date)
if(order_by_date_desc):
  statement = statement.order_by(TDAmeritradeDailyTable.c.Date.desc())

df_daily = pd.read_sql(statement, session.bind)

#Result
df_daily.head(30)

,Id,TickerId,Date,Open,High,Low,Close,Volume
0,4514430,766,2022-05-10,4035.18,4068.82,3958.17,4001.05,0.0
1,4514431,766,2022-05-11,3990.08,4049.09,3928.82,3935.18,0.0
2,4514432,766,2022-05-12,3903.95,3964.80,3858.87,3930.08,0.0
3,4514433,766,2022-05-13,3963.90,4038.88,3963.90,4023.89,0.0
4,4514434,766,2022-05-16,4013.02,4046.46,3983.99,4008.01,0.0
5,4514435,766,2022-05-17,4052.00,4090.72,4033.93,4088.85,0.0
6,4514436,766,2022-05-18,4051.98,4051.98,3911.91,3923.68,0.0
7,4514437,766,2022-05-19,3899.00,3945.96,3876.58,3900.79,0.0
8,4514438,766,2022-05-20,3927.76,3943.42,3810.32,3901.36,0.0
9,4514439,766,2022-05-23,3919.42,3981.88,3909.04,3973.75,0.0


**5. TDAmeritrade Minute Data**

In [ ]:
#Parameters
ticker_name = "AAPL"
start_date = "2023-01-23"
end_date = "2023-01-23"
order_by_date_desc = False
include_extended_hours = False

# Query
statement = select(TickerTable.c.Id).where(TickerTable.c.Symbol == ticker_name)
for row in conn.execute(statement):
    ticker_id = row[0]
statement = select(TDAmeritradeMinuteTable)
if(ticker_name != ""):
    statement = statement.where(TDAmeritradeMinuteTable.c.TickerId == ticker_id)
if(start_date != ""):
    statement = statement.where(TDAmeritradeMinuteTable.c.Date >= start_date)
if(end_date != ""):
    statement = statement.where(TDAmeritradeMinuteTable.c.Date <= end_date)
if(not include_extended_hours):
    statement = statement.where(TDAmeritradeMinuteTable.c.Time >= '09:30:00').where(TDAmeritradeMinuteTable.c.Time < '16:00:00')
if(order_by_date_desc):
  statement = statement.order_by(TDAmeritradeMinuteTable.c.Date.desc())
df_minute = pd.read_sql(statement, session.bind)

#Result
df_minute

,Id,TickerId,Date,Time,Open,High,Low,Close,Volume
0,68281619,45,2023-01-23,09:30:00,138.120,138.280,137.9100,138.1299,842538
1,68281620,45,2023-01-23,09:31:00,138.120,138.270,137.9000,138.1900,329740
2,68281621,45,2023-01-23,09:32:00,138.195,138.960,138.1800,138.8920,523487
3,68281622,45,2023-01-23,09:33:00,138.875,139.220,138.7800,139.2099,476255
4,68281623,45,2023-01-23,09:34:00,139.209,139.300,139.0200,139.0413,334055
...,...,...,...,...,...,...,...,...,...
385,68282004,45,2023-01-23,15:55:00,141.110,141.214,141.0147,141.1450,370106
386,68282005,45,2023-01-23,15:56:00,141.150,141.250,141.1100,141.2359,314479
387,68282006,45,2023-01-23,15:57:00,141.240,141.260,141.0400,141.1500,367945
388,68282007,45,2023-01-23,15:58:00,141.145,141.160,141.0300,141.0750,445446


**Aggregated Data**

In [ ]:
# Choose any time_frame value from the below. You can put any number in place of x
'''
x minutes - Minute 1, 5, 10, 15, 30, 60, 90, 120, 180, 240, 409,
x days - Daily 1 Day, 2 Day, 3 Day
microseconds
milliseconds
second
minute
hour
day
week
month
quarter
year
decade
century
millennium
'''

#Parameters
ticker_name = "AAPL"
time_frame = "5 minutes" 
table_name = "TDAmeritradeMinutePrice" # use "TDAmeritradeDailyPrice" for Daily Table or "TDAmeritradeMinutePrice" for Minute Table

In [ ]:
if(len(time_frame.split()) > 1):
  datetime_truncate = f'date_bin(\'{time_frame}\', "Date" + "Time", TIMESTAMP \'2001-01-01\')'
else:
  datetime_truncate = f'date_trunc(\'{time_frame}\', "Date")'

# Query
statement = select(TickerTable.c.Id).where(TickerTable.c.Symbol == ticker_name)
for row in conn.execute(statement):
    ticker_id = row[0]

statement = f'''
SELECT DISTINCT week as "DateTime", 
first_value("Open") OVER w as "Open", 
max("High") OVER w as "High",
min("Low") OVER w as "Low",
last_value("Close") OVER w as "Close",
sum("Volume") Over w as "Volume"
FROM (SELECT *, {datetime_truncate} as week
from public."{table_name}" where "TickerId" = {ticker_id}) as foo
WINDOW w AS (PARTITION BY week ORDER BY week)
order by week
'''

df_aggregated = pd.read_sql(statement, session.bind)
df_aggregated.insert(loc = 0, column = "Date", value = pd.to_datetime(df_aggregated['DateTime']).dt.date)
df_aggregated.insert(loc = 1, column = "Time", value = pd.to_datetime(df_aggregated['DateTime']).dt.time)
df_aggregated = df_aggregated.drop(columns=['DateTime'], axis =1)

In [ ]:
df_aggregated.head(25)

,Date,Time,Open,High,Low,Close,Volume
0,2022-05-27,09:30:00,146.0000,146.2300,145.2600,145.7900,3231233.0
1,2022-05-27,09:35:00,145.7900,146.6000,145.7900,146.4578,1895011.0
2,2022-05-27,09:40:00,146.4590,146.6600,145.8800,145.9601,1877028.0
3,2022-05-27,09:45:00,145.9700,147.1100,145.9400,146.9624,2177347.0
4,2022-05-27,09:50:00,146.9700,147.0450,146.5000,146.7700,1668313.0
5,2022-05-27,09:55:00,146.7600,147.3200,146.7250,147.0600,1302310.0
6,2022-05-27,10:00:00,147.0500,147.8000,146.9200,147.5580,1746047.0
7,2022-05-27,10:05:00,147.5400,147.7349,147.2900,147.6200,1435265.0
8,2022-05-27,10:10:00,147.6100,148.4050,147.5700,148.2950,1927643.0
9,2022-05-27,10:15:00,148.2982,148.6600,148.1300,148.4850,1569686.0
